In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import sys
sys.path.append('../')

In [3]:
from src.evaluation import Evaluator, Cross_validate_als
from src.utils import loading, Spark
from src.model_based import Als
from src.memory_based import Memory_based_CF

In [4]:
evaluators = [Evaluator('rmse'),Evaluator('accuracy')]

In [5]:
spark = Spark()

Spark UI address http://127.0.0.1:4042


In [24]:
datas = loading(spark, '../data/interim')

In [25]:
# ratings = spark.read.csv("../data/raw/ratings_small.csv",header=True)
# training, test = ratings.randomSplit([.8, .2])

In [26]:
training, test = datas['train_0.75_0.25'], datas['test_0.75_0.25']

In [27]:
training

DataFrame[userId: string, movieId: string, rating: string]

In [12]:
als = Als('userId', 'movieId', 'rating', .01, 0, 20)

In [13]:
tmp = Cross_validate_als(training, test, .2, [.1, .2], [10, 5], 0, evaluators)

  0%|          | 0/4 [00:00<?, ?it/s]


        training set num of rows 80557,
        test set num of rows 20279,
        training set num of users 610,
        training set num of movies 8983,
        test set num of users 610,
        test set num of movies 5065,
        


100%|██████████| 4/4 [00:33<00:00,  8.31s/it]


In [15]:
tmp

,rmse,accuracy
"(0.1, 10)",0.893562,0.680119
"(0.1, 5)",0.896979,0.680441
"(0.2, 10)",0.888640,0.653430
"(0.2, 5)",0.894260,0.654977


In [28]:
cf = Memory_based_CF(spark, 'user')

In [29]:
cf.fit(training)

In [30]:
prediction = cf.predict(training)

100%|██████████| 89977/89977 [01:36<00:00, 932.53it/s] 


In [31]:
evaluators[0].evaluate(prediction)

0.7588023855356845

In [32]:
evaluators[1].evaluate(prediction)

0.7873901108060949